In [14]:
# Welcome to your new notebook
# Type here in the cell editor to add code
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import *
import pandas as pd

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 16, Finished, Available, Finished)

###### **<mark>Reading Files from lakehouse using pandas</mark>**

In [15]:
df_sales = pd.read_excel('abfss://3e1f0f4e-4072-4fbe-86fe-4c969e139d91@onelake.dfs.fabric.microsoft.com/b2821490-f415-4bb0-846e-6cf5cc2fde87/Files/current/Sales*.xlsx',sheet_name='Sales')
df_return = pd.read_excel('abfss://3e1f0f4e-4072-4fbe-86fe-4c969e139d91@onelake.dfs.fabric.microsoft.com/b2821490-f415-4bb0-846e-6cf5cc2fde87/Files/current/Sales*.xlsx',sheet_name='Returns')

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 17, Finished, Available, Finished)

###### <mark>**Passing the pandas dataframe to spark**</mark>

In [16]:
df1 = spark.createDataFrame(df_sales)
df2 = spark.createDataFrame(df_return)

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 18, Finished, Available, Finished)

###### **<mark>Checking both first 5 rows of both datafram</mark>**

In [17]:
display(df1.head(5))
display(df2.head(5))

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1d844409-87f2-4c6b-96e0-0149083ce6bd)

SynapseWidget(Synapse.DataFrame, e006dfc9-b964-486e-8794-0105475c4df6)

###### <mark>**Joing both dataframes using Order_ID column**</mark>

###### <mark>**Since we needed only Return columns and the rest are being repeated so we are going to drop it**</mark>

In [18]:
final_df = df1.join(df2,df1.Order_ID==df2.Order_ID,how='left').drop(df2.Order_ID,df2.Customer_Name, df2.Sales_Amount)

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 20, Finished, Available, Finished)

###### <mark>**Adding Month, Year, Created_TS and Modified_TS columns to dataframe**</mark>

In [19]:
final_df = final_df.withColumns({"Order_year":year("Order_Date"),\
                    "Order_Month":month("Order_Date"),\
                    "Created_TS": current_timestamp(),\
                    "Modified_TS":current_timestamp(),\
                    })

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 21, Finished, Available, Finished)

###### <mark>**Creating View from final dataframe**</mark>

In [20]:
final_df.createOrReplaceTempView("ViewSales")

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 22, Finished, Available, Finished)

###### **<mark>Creating Bronze_sales table in our lakehouse</mark>**

In [21]:
%%sql
CREATE TABLE if NOT EXISTS bronze_sales
(
  Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Shipping_Cost 	float,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
USING DELTA
PARTITIONED BY (Order_Year,Order_Month)

StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 23, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

###### **<mark>Inserting data from ViewSales to bronze_sales table using Merge query</mark>**

In [22]:
%%sql
Merge into  Ecommerce_project.bronze_sales as BS
using ViewSales as VS
on  BS.Order_Year=VS.Order_Year and BS.Order_Month=VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then 
update SET
BS.Order_Date	=	VS.Order_Date	,
BS.Shipping_Date	=	VS.Shipping_Date	,
BS.Aging	=	VS.Aging	,
BS.Ship_Mode	=	VS.Ship_Mode	,
BS.Product_Category	=	VS.Product_Category	,
BS.Product	=	VS.Product	,
BS.Sales 	=	VS.Sales 	,
BS.Quantity	=	VS.Quantity	,
BS.Discount	=	VS.Discount	,
BS.Profit 	=	VS.Profit 	,
BS.Shipping_Cost 	=	VS.Shipping_Cost 	,
BS.Order_Priority	=	VS.Order_Priority	,
BS.Customer_ID	=	VS.Customer_ID	,
BS.Customer_Name	=	VS.Customer_Name	,
BS.Segment	=	VS.Segment	,
BS.City	=	VS.City	,
BS.State	=	VS.State	,
BS.Country	=	VS.Country	,
BS.Region  	=	VS.Region  	,
BS.Return  	=	VS.Return  	,
BS.Modified_TS	=	VS.Modified_TS	

when not matched then 
INSERT
(
BS.Order_ID,    
BS.Order_Date	,
BS.Shipping_Date	,
BS.Aging	,
BS.Ship_Mode	,
BS.Product_Category	,
BS.Product	,
BS.Sales 	,
BS.Quantity	,
BS.Discount	,
BS.Profit 	,
BS.Shipping_Cost 	,
BS.Order_Priority	,
BS.Customer_ID	,
BS.Customer_Name	,
BS.Segment	,
BS.City	,
BS.State	,
BS.Country	,
BS.Region  	,
BS.Return,
BS.Order_Year  	,
BS.Order_Month  	,
BS.Created_TS   	,
BS.Modified_TS	
)
values
(
VS.Order_ID,
VS.Order_Date	,
VS.Shipping_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Shipping_Cost 	,
VS.Order_Priority	,
VS.Customer_ID	,
VS.Customer_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Return,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TS   	,
VS.Modified_TS	

)


StatementMeta(, d046b5fa-8bcb-4225-9fd0-adfab4c7afea, 24, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>